<a href="https://colab.research.google.com/github/rajssnbeme/main/blob/main/sentimentJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [4]:
df = pd.read_csv('sentiments.csv')
df.columns = ["label", "text"]
x = df['text'].values
y = df['label'].values

In [5]:
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size=0.1, random_state=123)

In [13]:
x_train,y_train

(array(['recommend too', 'like it a lot', 'outstanding performance',
        "it's mediocre! not recommend", 'I really love it', 'It is rude',
        "It's really good", "It's really good", 'Definitely you should go',
        "I don't like this type", 'Recommend! I really enjoyed!',
        'poor performance'], dtype=object),
 array([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]))

In [6]:
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain = tokenizer.texts_to_sequences(x_train)
xtest = tokenizer.texts_to_sequences(x_test)

In [15]:
vocab_size = len(tokenizer.word_index)+1

maxlen = 10
xtrain = pad_sequences(xtrain, padding='post', maxlen=maxlen)
xtest = pad_sequences(xtest, padding='post', maxlen=maxlen)

print(x_train[3])
print(xtrain[3])

embedding_dim = 50
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))
model.add(layers.LSTM(units=50, return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=['accuracy'])
model.summary()
model.fit(xtrain, y_train, epochs=20, batch_size=16, verbose=False)



it's mediocre! not recommend
[ 3 23 12  5  0  0  0  0  0  0]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 50)            1500      
                                                                 
 lstm_2 (LSTM)               (None, 10, 50)            20200     
                                                                 
 lstm_3 (LSTM)               (None, 10)                2440      
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                           

In [18]:
loss,acc = model.evaluate(xtrain, y_train, verbose=False)
print("Training Accuracy: ", acc)
loss, acc = model.evaluate(xtest, y_test, verbose=False)
print("Test Accuracy: ", acc)

ypred = model.predict(xtest)

ypred[ypred > 0.5] = 1
ypred[ypred <= 0.5] = 0
cm = confusion_matrix(y_test, ypred)
print(cm)

result = zip(x_test, y_test, ypred)
for i in result:
    print(i)

Training Accuracy:  0.6666666865348816
Test Accuracy:  0.5
[[0 1]
 [0 1]]
('This is a great!', 1, array([1.], dtype=float32))
('Not good at all!', 0, array([1.], dtype=float32))
